In [ ]:
#tensorflow will takes over all the memory capacity during processing but computing on only hone GPU.
#GPU resources limitation
gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#0.333 is the percentage of Memory Capacity which will be used
config=tf.ConfigProto(gpu_options=gpu_options)
session = tf.Session(config=config)

In [2]:
#GPU id（0到n-1,其中n是机器上GPU卡的数量）
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

NameError: name 'A' is not defined

In [1]:
#single GPU demo
import tensorflow as tf
from tensorflow.contrib import slim
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
 
mnist=input_data.read_data_sets("/tmp/mnist/",one_hot=True)
 
num_gpus=2
num_steps=200
learning_rate=0.001
batch_size=1024
display_step=10
 
num_input=784
num_classes=10
def conv_net(x,is_training):
    # "updates_collections": None is very import ,without will only get 0.10
    batch_norm_params = {"is_training": is_training, "decay": 0.9, "updates_collections": None}
    #,'variables_collections': [ tf.GraphKeys.TRAINABLE_VARIABLES ]
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        activation_fn=tf.nn.relu,
                        weights_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
                        weights_regularizer=slim.l2_regularizer(0.0005),
                        normalizer_fn=slim.batch_norm, normalizer_params=batch_norm_params):
        with tf.variable_scope("ConvNet",reuse=tf.AUTO_REUSE):
            x = tf.reshape(x, [-1, 28, 28, 1])
            net = slim.conv2d(x, 6, [5,5], scope="conv_1")
            net = slim.max_pool2d(net, [2, 2],scope="pool_1")
            net = slim.conv2d(net, 12, [5,5], scope="conv_2")
            net = slim.max_pool2d(net, [2, 2], scope="pool_2")
            net = slim.flatten(net, scope="flatten")
            net = slim.fully_connected(net, 100, scope="fc")
            net = slim.dropout(net,is_training=is_training)
            net = slim.fully_connected(net, num_classes, scope="prob", activation_fn=None,normalizer_fn=None)
            return net
def train_single():
    X = tf.placeholder(tf.float32, [None, num_input])
    Y = tf.placeholder(tf.float32, [None, num_classes])
    logits=conv_net(X,True)
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits))
    opt=tf.train.AdamOptimizer(learning_rate)
    train_op=opt.minimize(loss)
    logits_test=conv_net(X,False)
    correct_prediction = tf.equal(tf.argmax(logits_test, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(1,num_steps+1):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(train_op,feed_dict={X:batch_x,Y:batch_y})
            if step%display_step==0 or step==1:
                loss_value,acc=sess.run([loss,accuracy],feed_dict={X:batch_x,Y:batch_y})
                print("Step:" + str(step) + ":" + str(loss_value) + " " + str(acc))
        print("Done")
        print("Testing Accuracy:",np.mean([sess.run(accuracy, feed_dict={X: mnist.test.images[i:i + batch_size],
              Y: mnist.test.labels[i:i + batch_size]}) for i in
              range(0, len(mnist.test.images), batch_size)]))
 
if __name__ == "__main__":
    train_single()

W0826 15:47:23.168834 140510538295104 deprecation.py:323] From <ipython-input-1-ce6c8aa764cb>:7: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0826 15:47:23.169934 140510538295104 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0826 15:47:23.170762 140510538295104 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be 

Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz


W0826 15:47:23.404805 140510538295104 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


W0826 15:47:23.854383 140510538295104 deprecation.py:323] From /home/se7ven/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0826 15:47:24.375272 140510538295104 deprecation.py:323] From <ipython-input-1-ce6c8aa764cb>:41: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Step:1:2.1768055 0.10058594
Step:10:1.7420014 0.111328125
Step:20:1.3150971 0.11230469
Step:30:0.9656034 0.11230469
Step:40:0.71995413 0.12792969
Step:50:0.54954606 0.29492188
Step:60:0.429086 0.8701172
Step:70:0.33470196 0.96972656
Step:80:0.29061007 0.97558594
Step:90:0.25628546 0.9765625
Step:100:0.21566352 0.98535156
Step:110:0.19937478 0.9785156
Step:120:0.18681864 0.97753906
Step:130:0.1634177 0.984375
Step:140:0.16077583 0.984375
Step:150:0.14618132 0.9863281
Step:160:0.14127463 0.9814453
Step:170:0.13692878 0.98535156
Step:180:0.115137175 0.98828125
Step:190:0.107293114 0.99121094
Step:200:0.10355579 0.98535156
Done
Testing Accuracy: 0.9869978


In [ ]:
#多GPU并行可分为模型并行和数据并行两大类,而其中数据并行又可分为同步方式和异步方式两种
#由于我们一般都会配置同样的显卡，因此这儿也选择了同步方式，也就是把数据分给不同的卡
#等所有的GPU都计算完梯度后进行平均，最后再更新梯度

#这里是数据部分 现在有多张卡 每张卡要分得不同的数据
#所以 batch_x,batch_y=mnist.train.next_batch(batch_size*num_gpus)
#这里保证每张卡都能获得batch_size大小的数据
#接下来切分数据 i为GPU索引 连续的batch_size大小数据分给同块卡
_x=X[i*batch_size:(i+1)*batch_size]
_y=Y[i*batch_size:(i+1)*batch_size]

#因为在多个GPU上共享同一张图 为防止名字混乱 用name_scope进行区分
        for i in range(2):
            with tf.device("/gpu:%d"%i):
                with tf.name_scope("tower_%d"%i):
                    _x=X[i*batch_size:(i+1)*batch_size]
                    _y=Y[i*batch_size:(i+1)*batch_size]
                    logits=conv_net(_x,dropout,reuse_vars,True)

#这里需要一个列表储存所有GPU的梯度，另外需要定义复用变量
tower_grads=[]
reuse_vars=False

#接下来开始计算各GPU的梯度
                    opt = tf.train.AdamOptimizer(learning_rate)
                    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_y,logits=logits))
                    grads=opt.compute_gradients(loss)
                    reuse_vars=True
                    tower_grads.append(grads)

#接下来求平均梯度
#Grads has been stored in tower_grads as [grads on gpu0, grads on gpu1]
#Notice that  zip(*) is basically transforming the above array
#to ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
#So that we can get for example the values of var0 on different GPUs
def average_gradients(tower_grads):
    average_grads=[]
    for grad_and_vars in zip(*tower_grads):
        grads=[]
        for g,_ in grad_and_vars:
            expend_g=tf.expand_dims(g,0)
            grads.append(expend_g)
        grad=tf.concat(grads,0)
        grad=tf.reduce_mean(grad,0)
        v=grad_and_vars[0][1]
        grad_and_var=(grad,v)
        average_grads.append(grad_and_var)
    return average_grads

#接下来更新梯度
grads=average_gradients(tower_grads)
train_op=opt.apply_gradients(grads)

In [2]:
#multiple GPU demo
import time
import numpy as np
 
import tensorflow as tf
from tensorflow.contrib import slim
from tensorflow.examples.tutorials.mnist import input_data
 
mnist = input_data.read_data_sets("mnist/", one_hot=True)


def get_available_gpus():
    from tensorflow.python.client import device_lib as _device_lib
    local_device_protos = _device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
 
num_gpus = len(get_available_gpus())
print("Available GPU Number :"+str(num_gpus))
 
num_steps = 1000
learning_rate = 0.001
batch_size = 1000
display_step = 10
 
num_input = 784
num_classes = 10
 
def conv_net_with_layers(x,is_training,dropout = 0.75):
    with tf.variable_scope("ConvNet", reuse=tf.AUTO_REUSE):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = tf.layers.conv2d(x, 12, 5, activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, 2, 2)
        x = tf.layers.conv2d(x, 24, 3, activation=tf.nn.relu)
        x = tf.layers.max_pooling2d(x, 2, 2)
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, 100)
        x = tf.layers.dropout(x, rate=dropout, training=is_training)
        out = tf.layers.dense(x, 10)
        out = tf.nn.softmax(out) if not is_training else out
    return out
 
def conv_net(x,is_training):
    # "updates_collections": None is very import ,without will only get 0.10
    batch_norm_params = {"is_training": is_training, "decay": 0.9, "updates_collections": None}
    #,'variables_collections': [ tf.GraphKeys.TRAINABLE_VARIABLES ]
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        activation_fn=tf.nn.relu,
                        weights_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01),
                        weights_regularizer=slim.l2_regularizer(0.0005),
                        normalizer_fn=slim.batch_norm, normalizer_params=batch_norm_params):
        with tf.variable_scope("ConvNet",reuse=tf.AUTO_REUSE):
            x = tf.reshape(x, [-1, 28, 28, 1])
            net = slim.conv2d(x, 6, [5,5], scope="conv_1")
            net = slim.max_pool2d(net, [2, 2],scope="pool_1")
            net = slim.conv2d(net, 12, [5,5], scope="conv_2")
            net = slim.max_pool2d(net, [2, 2], scope="pool_2")
            net = slim.flatten(net, scope="flatten")
            net = slim.fully_connected(net, 100, scope="fc")
            net = slim.dropout(net,is_training=is_training)
            net = slim.fully_connected(net, num_classes, scope="prob", activation_fn=None,normalizer_fn=None)
            return net
 
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            expend_g = tf.expand_dims(g, 0)
            grads.append(expend_g)
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads
 
PS_OPS = ['Variable', 'VariableV2', 'AutoReloadVariable']
def assign_to_device(device, ps_device='/cpu:0'):
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return "/" + ps_device
        else:
            return device
 
    return _assign
 
def trainDouble():
    with tf.device("/cpu:0"):
        global_step=tf.train.get_or_create_global_step()
        tower_grads = []
        X = tf.placeholder(tf.float32, [None, num_input])
        Y = tf.placeholder(tf.float32, [None, num_classes])
        opt = tf.train.AdamOptimizer(learning_rate)
        with tf.variable_scope(tf.get_variable_scope()):
            for i in range(num_gpus):
                with tf.device(assign_to_device('/gpu:{}'.format(i), ps_device='/cpu:0')):
                    _x = X[i * batch_size:(i + 1) * batch_size]
                    _y = Y[i * batch_size:(i + 1) * batch_size]
                    logits = conv_net(_x, True)
                    tf.get_variable_scope().reuse_variables()
                    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=_y, logits=logits))
                    grads = opt.compute_gradients(loss)
                    tower_grads.append(grads)
                    if i == 0:
                        logits_test = conv_net(_x, False)
                        correct_prediction = tf.equal(tf.argmax(logits_test, 1), tf.argmax(_y, 1))
                        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        grads = average_gradients(tower_grads)
        train_op = opt.apply_gradients(grads)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for step in range(1, num_steps + 1):
                batch_x, batch_y = mnist.train.next_batch(batch_size * num_gpus)
                ts = time.time()
                sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
                te = time.time() - ts
                if step % 10 == 0 or step == 1:
                    loss_value, acc = sess.run([loss, accuracy], feed_dict={X: batch_x, Y: batch_y})
                    print("Step:" + str(step) + ":" + str(loss_value) + " " + str(acc)+", %i Examples/sec" % int(len(batch_x)/te))
            print("Done")
            print("Testing Accuracy:",
                  np.mean([sess.run(accuracy, feed_dict={X: mnist.test.images[i:i + batch_size],
                                                         Y: mnist.test.labels[i:i + batch_size]}) for i in
                           range(0, len(mnist.test.images), batch_size)]))
 
if __name__ == "__main__":
    #train_single()
    trainDouble()

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Available GPU Number :2
Step:1:2.1677146 0.087, 12108 Examples/sec
Step:10:1.6895834 0.102, 142653 Examples/sec
Step:20:1.275157 0.121, 127871 Examples/sec
Step:30:0.9419908 0.848, 164954 Examples/sec
Step:40:0.66217387 0.959, 162070 Examples/sec
Step:50:0.50352776 0.974, 155658 Examples/sec
Step:60:0.38034534 0.97, 139347 Examples/sec
Step:70:0.3190288 0.983, 162939 Examples/sec
Step:80:0.26566234 0.984, 138118 Examples/sec
Step:90:0.21712294 0.981, 169875 Examples/sec
Step:100:0.20165718 0.991, 169645 Examples/sec
Step:110:0.16454723 0.989, 156995 Examples/sec
Step:120:0.16735114 0.987, 160513 Examples/sec
Step:130:0.1390191 0.987, 162098 Examples/sec
Step:140:0.12545046 0.986, 165828 Examples/sec
Step:150:0.14234646 0.988, 169518 Examples/sec
Step:160:0.1269624 0.992, 155650 Examples/sec
Step:170:0.1038568 0.992